In [1]:
from ner_seg import ClassificationModel, TokenizerModel
from transformers import pipeline
import torch



/home/carlinet/.local/share/virtualenvs/processor-ner-U0FrUamN/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
from datasets import load_from_disk
ds = load_from_disk("/home/carlinet/repos/soduco/paper-entryseg-icdar23-code/dataset/training/02-xp-datasets/exp_labelledtext_breaks_indents_left_diff_right_ref/")
print(ds["train"][3])

{'input_ids': [26918, 7, 12999, 26, 2093, 81, 15906, 8, 13, 285, 7, 32023, 32028, 32007, 2445, 26, 185, 26, 28013, 7, 6928, 9, 32005, 32028, 32030, 26918, 7, 15097, 11084, 7, 772, 291, 105, 26, 9567, 9878], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels_str': ['I-PER', 'O', 'I-ACT', 'I-ACT', 'I-ACT', 'I-ACT', 'I-ACT', 'I-ACT', 'I-ACT', 'I-ACT', 'O', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'I-CARDINAL', 'O', 'I-EEND', 'O', 'I-EBEGIN', 'I-PER', 'O', 'I-ACT', 'I-ACT', 'O', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC'], 'labels': [2, 0, 6, 6, 6, 6, 6, 6, 6, 6, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 5, 0, 10, 0, 9, 2, 0, 6, 6, 0, 1, 1, 1, 1, 1, 1], 'tokens': ['▁Moreau', ',', '▁architecte', '-', 'vé', 'r', 'ificateur', '▁de', '▁la', '▁ville', ',', '<rhspace-1>', '<break>', '<lhspace-3>', '▁Monsieur', '-', 'le', '-', 'Prince', ',', '▁58', '.', '<rhspace-3>', '<break>', 

In [36]:
id=5
v = zip(ds["train"][id]["tokens"], ds["train"][id]["labels_str"])
print(list(v))

[('<rhspace-1>', 'I-ACT'), ('<break>', 'I-ACT'), ('<lhspace-2>', 'I-ACT'), ('▁Se', 'I-ACT'), ('dan', 'I-ACT'), (',', 'I-ACT'), ('▁', 'I-ACT'), ('exportation', 'I-ACT'), (',', 'O'), ('▁Jeune', 'I-LOC'), ('urs', 'I-LOC'), (',', 'O'), ('<rhspace-1>', 'I-CARDINAL'), ('<break>', 'I-CARDINAL'), ('<lhspace-2>', 'I-CARDINAL'), ('▁46', 'I-CARDINAL'), ('.', 'O'), ('<rhspace-3>', 'I-EEND'), ('<break>', 'O'), ('<lhspace-1>', 'I-EBEGIN'), ('▁Du', 'I-PER'), ('ques', 'I-PER'), ('ne', 'I-PER'), ('▁et', 'I-PER'), ('▁Cie', 'I-PER'), (',', 'O'), ('▁en', 'I-ACT'), ('▁farine', 'I-ACT'), (',', 'O'), ('<rhspace-1>', 'I-LOC'), ('<break>', 'I-LOC'), ('<lhspace-3>', 'I-LOC'), ('▁Co', 'I-LOC'), ('quill', 'I-LOC'), ('ière', 'I-LOC'), (',', 'O'), ('▁32', 'I-CARDINAL'), ('.', 'O'), ('<rhspace-3>', 'I-EEND'), ('<break>', 'O'), ('<lhspace-1>', 'I-EBEGIN'), ('▁Dur', 'I-PER')]


In [43]:
ner = pipeline("ner", model=ClassificationModel.classification_model, tokenizer=TokenizerModel.layout_tokenizer, aggregation_strategy="simple")

In [44]:
text = "<lhspace-2>Béguin, orfévre, fab. de couverts, Jeûneurs,<rhspace-1><break>" +\
       "<lhspace-2>127 rue des lilas<rhspace-3><break>"

doc = ner(text)
#inputs = TokenizerModel.layout_tokenizer(text, return_tensors="pt", padding="longest")

In [46]:
#with torch.no_grad():
#   logits = ClassificationModel.classification_model(**inputs).logits
#predictions = torch.argmax(logits, dim=2)
doc

[{'entity_group': 'PER',
  'score': 0.999197,
  'word': 'Béguin',
  'start': 11,
  'end': 17},
 {'entity_group': 'ACT',
  'score': 0.9995751,
  'word': 'orfévre, fab. de couverts',
  'start': 18,
  'end': 44},
 {'entity_group': 'LOC',
  'score': 0.99981314,
  'word': 'Jeûneurs',
  'start': 45,
  'end': 54},
 {'entity_group': 'CARDINAL',
  'score': 0.9979583,
  'word': '<rhspace-1><break><lhspace-2> 127',
  'start': 55,
  'end': 87},
 {'entity_group': 'LOC',
  'score': 0.9996253,
  'word': 'rue des lilas',
  'start': 87,
  'end': 101},
 {'entity_group': 'CARDINAL',
  'score': 0.4030483,
  'word': '<rhspace-3>',
  'start': 101,
  'end': 112}]

In [41]:
tokens = TokenizerModel.layout_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
predicted_token_class = [ClassificationModel.classification_model.config.id2label[t.item()] for t in predictions[0]]
list(zip(tokens, predicted_token_class))

[('<s>', 'O'),
 ('<lhspace-2>', 'O'),
 ('▁Bé', 'I-PER'),
 ('guin', 'I-PER'),
 (',', 'O'),
 ('▁or', 'I-ACT'),
 ('févr', 'I-ACT'),
 ('e', 'I-ACT'),
 (',', 'I-ACT'),
 ('▁fa', 'I-ACT'),
 ('b', 'I-ACT'),
 ('.', 'I-ACT'),
 ('▁de', 'I-ACT'),
 ('▁couverts', 'I-ACT'),
 (',', 'O'),
 ('▁Je', 'I-LOC'),
 ('û', 'I-LOC'),
 ('neur', 'I-LOC'),
 ('s', 'I-LOC'),
 (',', 'O'),
 ('<rhspace-1>', 'I-CARDINAL'),
 ('<break>', 'I-CARDINAL'),
 ('<lhspace-2>', 'I-CARDINAL'),
 ('▁127', 'I-CARDINAL'),
 ('▁rue', 'I-LOC'),
 ('▁des', 'I-LOC'),
 ('▁l', 'I-LOC'),
 ('il', 'I-LOC'),
 ('as', 'I-LOC'),
 ('<rhspace-3>', 'I-CARDINAL'),
 ('<break>', 'O'),
 ('</s>', 'O')]

In [10]:
predicted_token_class

['O',
 'I-EBEGIN',
 'I-PER',
 'I-PER',
 'O',
 'I-ACT',
 'I-ACT',
 'I-ACT',
 'I-ACT',
 'I-ACT',
 'I-ACT',
 'I-ACT',
 'I-ACT',
 'I-ACT',
 'O',
 'I-LOC',
 'I-LOC',
 'I-LOC',
 'I-LOC',
 'O',
 'I-CARDINAL',
 'O',
 'I-EBEGIN']

In [2]:
%load_ext autoreload
%autoreload 2

In [12]:
from ner_seg import InputStreamGenerator

In [38]:
seq = InputStreamGenerator.InputLineStreamGenerator(["/tmp/0701.json"])
texts = InputStreamGenerator.TextEntityClassification(seq)


In [39]:
for i,t in enumerate(texts):
    print(f"Texte #{i}")
    print("\n".join(t))



Texte #0
<ACT>pot, broches de cuisinières, et tout
ce qui concerne cette partie</ACT>, <LOC>Tem-
ple</LOC>, <CARDINAL>69</CARDINAL>.<EEND></EEND>
<EBEGIN></EBEGIN><PER>Caron (P.</PER>), <ACT>ingénieur-mécanicien</ACT>,
ci-devant <LOC>Faub. -St-Martin</LOC>, <CARDINAL>147</CARDINAL>,
actuellement, <LOC>même rue</LOC>, <CARDINAL>168</CARDINAL>;
fabrique hydro-extracteur, dit<ACT>
se</ACT>choir me<ACT>canique, sans feu ni
pression, avec mouvement pro-
gressif; metiers à couvrir la gui-
pure, cartisanne, laine, gomme,
trait d'or et d'argent,</ACT> <LOC>milanaise</LOC><ACT>,
etc</ACT>. <ACT>Ces métiers remplacent les
rouets de Lyon</ACT>, auxquels on
peut adapter ce nouveau système.<ACT>
Métiers continus, à doubler et à
retordre, à l'usage des passe-
mentiers, retordeurs, fabricants
de fils retors et autres; engre-</ACT>
<ACT>nage</ACT>s el<ACT>astiques</ACT> résistant à une
Texte #1
<ACT>retordre, à l'usage des passe-
mentiers, retordeurs, fabricants
de fils retors et autres; engre-
nages 

In [40]:
from ner_seg import ClassificationModel

In [ ]:
ClassificationMo